In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [5]:
dset = pd.read_csv('bank_csv.csv')

In [6]:
dset.head()

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,1,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,2,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,3,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,4,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,5,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [7]:
#We drop the unnamed column as it provides no value, it represents the column number
#We drop the duration column because of the instructions in the UCI repository (Cloumn should be discarded in order to have a realistic predictive model)
dset.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
dset.drop(["a"], axis=1, inplace=True)
dset.drop(["duration"], axis=1, inplace=True) 

In [8]:
 # Change missing values to NaN so they can be dropped
data = dset.replace("unknown", np.NaN)

In [9]:
data.dropna(inplace=True)

In [10]:
 #Display column formats
data.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [11]:
obj_data = data.select_dtypes(include=['object']).copy() # Make a table with only the categorical variables for easier visualisation
obj_data.head()

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,no
2,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent,no
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,no
4,services,married,high.school,no,no,yes,telephone,may,mon,nonexistent,no
6,admin.,married,professional.course,no,no,no,telephone,may,mon,nonexistent,no


In [12]:
data["job"].value_counts()

admin.           8737
blue-collar      5675
technician       5473
services         2857
management       2311
retired          1216
self-employed    1092
entrepreneur     1089
unemployed        738
housemaid         690
student           610
Name: job, dtype: int64

In [13]:
data["marital"].value_counts()

married     17492
single       9443
divorced     3553
Name: marital, dtype: int64

In [14]:
data["education"].value_counts()

university.degree      10412
high.school             7699
professional.course     4321
basic.9y                4276
basic.4y                2380
basic.6y                1389
illiterate                11
Name: education, dtype: int64

In [15]:
# Label encode education as it can be seen as an order
data["education"] = data["education"].astype('category')
data["education"] = data["education"].cat.codes

In [16]:
data["month"] = data["month"].astype('category') 
data["month"] = data["month"].cat.codes

In [17]:
data["day_of_week"] = data["day_of_week"].astype('category')
data["day_of_week"] = data["day_of_week"].cat.codes

In [18]:
#Change the data to binary to improve readability
data = data.replace("no", 0) 
data = data.replace("yes", 1)

In [19]:
#Custom Binary encoding marital status (One income/two income households)
data["marital"] = np.where(data["marital"].str.contains("married"), 1, 0) 

In [20]:
data["pdays"].value_counts()

999    29178
3        381
6        363
4        102
2         53
9         53
7         50
12        50
5         43
10        40
13        33
11        25
15        22
1         21
14        17
0         14
8         13
16         8
17         6
18         5
22         3
19         3
21         2
25         1
26         1
27         1
Name: pdays, dtype: int64

In [21]:
data["previous"].value_counts()

0    25836
1     3752
2      633
3      190
4       56
5       16
6        4
7        1
Name: previous, dtype: int64

In [22]:
data = pd.get_dummies(data, columns=["job", "contact","poutcome"], prefix=["job", "contact","poutcome"]) #One-hot encoding

In [23]:
data

,age,marital,education,default,housing,loan,month,day_of_week,campaign,pdays,...,job_self-employed,job_services,job_student,job_technician,job_unemployed,contact_cellular,contact_telephone,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,1,0,0,0,0,6,1,1,999,...,0,0,0,0,0,0,1,0,1,0
2,37,1,3,0,1,0,6,1,1,999,...,0,1,0,0,0,0,1,0,1,0
3,40,1,1,0,0,0,6,1,1,999,...,0,0,0,0,0,0,1,0,1,0
4,56,1,3,0,0,1,6,1,1,999,...,0,1,0,0,0,0,1,0,1,0
6,59,1,5,0,0,0,6,1,1,999,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,5,0,1,0,7,0,1,999,...,0,0,0,0,0,1,0,0,1,0
41184,46,1,5,0,0,0,7,0,1,999,...,0,0,0,0,0,1,0,0,1,0
41185,56,1,6,0,1,0,7,0,2,999,...,0,0,0,0,0,1,0,0,1,0
41186,44,1,5,0,0,0,7,0,1,999,...,0,0,0,1,0,1,0,0,1,0


In [30]:
data["y"].value_counts()

0    26629
1     3859
Name: y, dtype: int64

We have a massively unbalanced dataset, with less than 15% of the entries being positive outcomes. We will therefore randomly under-sample the negative cases to balance the dataset.

In [24]:
from imblearn.under_sampling import RandomUnderSampler

In [25]:
# Create the under sampler
under_sampler = RandomUnderSampler()

In [26]:
# Split the data into feature and target variables
X = data.drop(['y'], axis=1) # assuming 'target_col' is the name of your target column
y = data['y']

In [27]:
# Fit and transform the data using the under sampler
X_under, y_under = under_sampler.fit_resample(X, y)

In [28]:
merged_df = pd.concat([X_under, y_under], axis=1)

In [29]:
merged_df

,age,marital,education,default,housing,loan,month,day_of_week,campaign,pdays,...,job_services,job_student,job_technician,job_unemployed,contact_cellular,contact_telephone,poutcome_failure,poutcome_nonexistent,poutcome_success,y
0,30,0,3,0,1,0,0,4,2,999,...,1,0,0,0,1,0,0,1,0,0
1,29,1,6,0,0,1,4,0,2,999,...,1,0,0,0,1,0,0,1,0,0
2,31,0,5,0,1,0,6,2,5,999,...,0,0,0,0,1,0,1,0,0,0
3,33,1,5,0,1,0,1,0,2,999,...,0,0,0,0,1,0,0,1,0,0
4,32,0,2,0,1,0,6,3,2,999,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7713,62,1,6,0,1,0,7,2,1,1,...,0,0,0,0,1,0,0,0,1,1
7714,62,1,6,0,0,0,7,2,2,6,...,0,0,0,0,1,0,0,0,1,1
7715,37,1,6,0,1,0,7,0,1,999,...,0,0,0,0,1,0,0,1,0,1
7716,73,1,5,0,1,0,7,0,1,999,...,0,0,0,0,1,0,0,1,0,1


In [31]:
p_data = merged_df